In [21]:
import numpy as np
import pandas as pd
import sys
sys.path.append("/home/shizhenkun/codebase/BioUniprot/tools/")
import funclib
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import linear_model, datasets
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
def load_file(path):
    train = pd.read_hdf(path + 'train.h5', key='data')
    train.ec_number = train.ec_number.apply(lambda x: x.replace(',', ''))
    test = pd.read_hdf(path + 'test.h5', key='data')
    test.ec_number = test.ec_number.apply(lambda x: x.replace(',', ''))
    head = funclib.table_head + ['f'+str(i) for i in range(1, 1901)]
    train.columns = head
    test.columns = head
    return train, test

def xgmain(X_train_std, Y_train, X_test_std, Y_test):
    model = XGBClassifier(objective='binary:logistic', random_state=42, use_label_encoder=False, n_jobs=-2, eval_metric='mlogloss')
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

def lrmain(X_train_std, Y_train, X_test_std, Y_test):
    logreg = linear_model.LogisticRegression(solver = 'lbfgs', n_jobs=-2)
    logreg.fit(X_train_std, Y_train)
    predict = logreg.predict(X_test_std)
    lrpredpro = logreg.predict_proba(X_test_std)
    groundtruth = Y_test
    predictprob = lrpredpro[:,1]
    return groundtruth, predict, predictprob

def caculate_metrics(groundtruth, predict, predictprob):
    acc = metrics.accuracy_score(groundtruth, predict)
    precision = metrics.precision_score(groundtruth, predict, zero_division=1 )
    recall = metrics.recall_score(groundtruth, predict)
    f1 = metrics.f1_score(groundtruth, predict)
    auroc = metrics.roc_auc_score(groundtruth, predictprob)
    auprc = metrics.average_precision_score(groundtruth, predictprob)
    tn, fp, fn, tp = metrics.confusion_matrix(groundtruth, predict).ravel()

    npv = tn/(fn+tn+1.4E-45)
    print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
    print('xgboost', '\t%f' %acc,'\t%f'% precision,'\t\t%f'%npv,'\t%f'% recall,'\t%f'% f1, '\t%f'% auroc,'\t%f'% auprc, '\t', 'tp:',tp,'fp:',fp,'fn:',fn,'tn:',tn)
    
def dtmain(X_train_std, Y_train, X_test_std, Y_test):
    model = tree.DecisionTreeClassifier()
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

def rfmain(X_train_std, Y_train, X_test_std, Y_test):
    model = RandomForestClassifier(oob_score=True, random_state=10, n_jobs=-2)
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

def gbdtmain(X_train_std, Y_train, X_test_std, Y_test):
    model = GradientBoostingClassifier(random_state=10)
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

In [3]:
ori_file_dir = '/home/shizhenkun/codebase/BioUniprot/tasks/task1/data/'
train, test = load_file(ori_file_dir)

In [5]:
res_data=funclib.getblast(train,test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [6]:
isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
res_data['diamoion_pred'] = res_data['sseqid'].apply(lambda x: isEmzyme_dict.get(x))

blast_res = pd.DataFrame
blast_res = res_data[['id','pident','bitscore', 'diamoion_pred']]

X_train = train.iloc[:,12:]
X_test = test.iloc[:,12:]
Y_train = train.iloc[:,2].astype('int')
Y_test = test.iloc[:,2].astype('int')
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).flatten()
Y_test = np.array(Y_test).flatten()

In [ ]:
groundtruth_lr, predict_lr, predictprob_lr = lrmain (X_train, Y_train, X_test, Y_test)

In [24]:
groundtruth_xg, predict_xg, predictprob_xg = xgmain(X_train, Y_train, X_test, Y_test)

In [ ]:
groundtruth_dt, predict_dt, predictprob_dt = dtmain(X_train, Y_train, X_test, Y_test)

In [ ]:
groundtruth_rf, predict_rf, predictprob_rf = rfmain(X_train, Y_train, X_test, Y_test)

In [ ]:
groundtruth_gbdt, predict_gbdt, predictprob_gbdt = gbdtmain(X_train, Y_train, X_test, Y_test)

In [20]:
train[~train.isemzyme & (train.ec_number!='-')]

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
1735,P68313,ERTS_RAT,False,False,1,6.-.-.-,1,1986-07-21,1986-07-21,2021-04-07,...,0.043036,-0.024681,-0.273989,0.032162,-0.064190,0.005630,0.066531,0.053463,0.034046,-0.011738
1153,P00646,CEA3_ECOLX,False,False,1,3.1.-.-,2,1986-07-21,1988-04-01,2021-04-07,...,0.011701,0.004105,-0.018341,0.017004,-0.069992,-0.063424,-0.012459,0.003463,0.031870,-0.036516
1162,P00645,CEAC_ECOLX,False,False,1,3.1.-.-,2,1986-07-21,1989-10-01,2021-04-07,...,0.012196,0.005072,-0.040222,0.018408,-0.052703,-0.076993,-0.001214,0.008080,0.005036,-0.048188
1154,P03633,SCAFB_BPPHS,False,False,1,3.4.-.-,2,1986-07-21,1986-07-21,2021-04-07,...,0.027132,0.266320,-0.429699,0.015056,-0.047524,-0.024693,0.251561,-0.000288,0.039648,-0.008624
1161,P03433,PA_I34A1,False,False,1,3.1.-.-,2,1986-07-21,2007-10-02,2021-04-07,...,0.001511,-0.023552,-0.186022,0.005494,-0.246220,0.143945,0.151142,0.051355,-0.058095,-0.006349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406748,O34581,YOQL_BACSU,False,False,1,3.1.-.-,2,2009-11-24,1998-01-01,2021-04-07,...,0.015478,0.353042,-0.098861,0.067722,0.080912,-0.047497,0.065789,-0.004080,0.086778,0.338710
406749,O34973,YTAP_BACSU,False,False,1,3.-.-.-,1,2009-11-24,1998-01-01,2021-04-07,...,0.211230,-0.006704,-0.501742,-0.017686,-0.271717,0.496858,-0.148190,-0.318121,-0.039267,0.007634
406751,O64175,YOSQ_BPSPB,False,False,1,3.1.-.-,2,2009-11-24,1998-08-01,2020-08-12,...,-0.016808,0.165690,-0.098466,0.013183,-0.076029,0.052880,0.342779,0.031015,0.054560,-0.021438
406752,A9R2J1,XNI_YERPG,False,False,1,3.1.-.-,2,2009-11-24,2008-02-05,2021-04-07,...,0.043425,0.254228,-0.039421,-0.172320,-0.276798,-0.021701,-0.005170,0.005355,-0.092833,-0.144663


In [23]:
res_data

,id,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,diamoion_pred
0,B0VRF9,Q0VPJ1,57.8,256,107,1,44,298,32,287,1.600000e-102,304.0,True
1,C6E8M5,P29923,59.0,100,40,1,1,99,1,100,4.800000e-32,110.0,True
2,Q3SWM0,B5EAK5,48.7,279,142,1,1,278,1,279,7.640000e-91,275.0,True
3,O44410,Q9DB85,47.9,219,109,3,128,343,241,457,6.150000e-62,207.0,True
4,Q2QSR8,Q9LV33,58.9,470,187,3,26,489,39,508,1.660000e-210,594.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34808,Q06GJ0,P49010,73.6,595,155,1,1,593,1,595,0.000000e+00,958.0,True
34809,O17679,Q9VYD1,32.3,189,95,8,432,618,1341,1498,1.370000e-15,84.7,True
34810,A0A193AU77,Q9MB73,77.4,478,107,1,1,477,1,478,2.120000e-288,793.0,True
34811,A0A482AS24,O94409,70.6,412,76,2,1,411,1,368,5.210000e-182,513.0,False


In [ ]:
methods=['lr', 'xg', 'dt', 'rf', 'gbdt']
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
for method in methods:
    funclib.get_integrated_results(res_data, train, test, method)

In [7]:
isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
res_data['diamoion_pred'] = res_data['sseqid'].apply(lambda x: isEmzyme_dict.get(x))

blast_res = pd.DataFrame
blast_res = res_data[['id','pident','bitscore', 'diamoion_pred']]

X_train = train.iloc[:,12:]
X_test = test.iloc[:,12:]
Y_train = train.iloc[:,2].astype('int')
Y_test = test.iloc[:,2].astype('int')
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).flatten()
Y_test = np.array(Y_test).flatten()

In [ ]:
funclib.run_integrated(res_data, train, test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix


0it [00:00, ?it/s]/home/shizhenkun/codebase/BioUniprot/tools/funclib.py:236: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else:
/home/shizhenkun/codebase/BioUniprot/tools/funclib.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
46129it [00:24, 1920.05it/s]


baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
xg 	 0.935615  	0.919272 		 0.949210  	0.937716	 0.928403		 	 	 	 tp:19256  fp:1691  fn:1279  tn:23903


In [ ]:
aa